In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras1 deeppavlov tensorflow-gpu==1.14.0rc1

In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    #'multi_cased_L-12_H-768_A-12.zip',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    #'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)
tf.keras.utils.get_file(
    fname='paraphraser_rubert_v0.tar.gz',
    #'paraphraser_rubert_v0.tar.gz',
    origin='http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz',
    #'http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_bert_v0.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

1.14.0-rc1


'./models/paraphraser_rubert_v0.tar.gz'

In [3]:
# some constants to construct input tensors
BATCH_SIZE = 4
MAX_LEN = 512

input_ids = np.array([[101, 142] + [43279, 38615, 23742, 23685] * (MAX_LEN // 4 - 1) + [4242, 102],
                      [101] + [3351, 32132, 64233] * (MAX_LEN // 4 - 1) + [142, 102] + [0] * (MAX_LEN // 4)] * (BATCH_SIZE // 2))
input_mask = np.int32(np.not_equal(input_ids, 0))

transparent_input_mask = np.ones_like(input_ids)

sep_ids = np.int32(np.equal(input_ids, 102))

token_types = np.cumsum(sep_ids, axis=1) - sep_ids

labels = np.array([1, 0] * (BATCH_SIZE // 2))

# Refactored implementation

In [4]:
from bert_dp import bert
keras_graph = tf.compat.v1.Graph()

In [5]:
with keras_graph.as_default():
    b_ = bert.BERT(name='bert')
    keras_sess = tf.compat.v1.Session()
#     inp = tf.keras.layers.Input(shape=(None,), batch_size=BATCH_SIZE, dtype=tf.int32)
    po_ = b_(input_ids)  # b_(inp)
    output_weights_ = tf.get_variable(
        "output_weights", [2, 768],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias_ = tf.get_variable(
        "output_bias", [2], initializer=tf.zeros_initializer())

W0611 21:44:27.215738 139906068133696 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:30: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0611 21:44:27.218014 139906068133696 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0611 21:44:27.231462 139906068133696 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instru

In [6]:
with keras_graph.as_default():
    with tf.compat.v1.variable_scope("loss", reuse=tf.compat.v1.AUTO_REUSE):
        output_layer_ = po_  # tf.nn.dropout(po_, keep_prob=0.5)
        logits_ = tf.matmul(output_layer_, output_weights_, transpose_b=True)
        logits_ = tf.nn.bias_add(logits_, output_bias_)

        one_hot_labels_ = tf.one_hot(tf.constant(labels), depth=2, dtype=tf.float32)

        y_predictions_ = tf.argmax(logits_, axis=-1)
#         if not False:  # self.multilabel:
        log_probs_ = tf.nn.log_softmax(logits_, axis=-1)
        y_probas_ = tf.nn.softmax(logits_, axis=-1)
        per_example_loss_ = -tf.reduce_sum(one_hot_labels_ * log_probs_, axis=-1)
        loss_ = tf.reduce_mean(per_example_loss_)
#         else:
#             y_probas_ = tf.nn.sigmoid(logits_)
#             loss_ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=one_hot_labels_, logits=logits_))

In [7]:
keras_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32>,
 <tf.

In [8]:
with keras_graph.as_default():
    from tensorflow.python.ops import variables
    keras_vs = variables._all_saveable_objects()
    keras_saver = tf.compat.v1.train.Saver(keras_vs)
    keras_sess.run(tf.global_variables_initializer())
    keras_saver_checkpoint_path = 'models/paraphraser_rubert/model_rubert' # 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
    # 'models/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
    keras_saver.restore(keras_sess, keras_saver_checkpoint_path)

W0611 21:44:42.983929 139906068133696 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [ ]:
with keras_graph.as_default():
    # object-based loading/saving... not currently working
#     checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
#     checkpoint = tf.train.Checkpoint(model=b_)
#     status = checkpoint.restore(checkpoint_path).assert_consumed().initializer_or_restore(session=keras_sess)
    # conversion for future usage
    b_.save_weights('models/rubert_cased_L-12_H-768_A-12_v1_converted/bert_model'
#                      'models/bert_multi_cased_L-12_H-768_A-12_converted/bert_model'
    )

# Original implementation

In [9]:
from bert_dp.modeling import BertModel, BertConfig
orig_graph = tf.compat.v1.Graph()

In [10]:
with orig_graph.as_default():

    input_ids_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='ids_ph')
    input_masks_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='masks_ph')
    token_types_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='token_types_ph')

    orig_sess = tf.compat.v1.Session()
    b = BertModel(config=BertConfig.from_json_file('models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json'),
                  # 'models/bert_multi_cased_L-12_H-768_A-12/bert_config.json'
                  is_training=False,
                  input_ids=input_ids_ph,
                  input_mask=input_masks_ph,
                  token_type_ids=token_types_ph,
                  use_one_hot_embeddings=False
                  )
    po = b.get_pooled_output()
    output_weights = tf.get_variable(
        "output_weights", [2, 768],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [2], initializer=tf.zeros_initializer())

W0611 21:44:57.483940 139906068133696 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0611 21:44:57.489105 139906068133696 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:178: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0611 21:44:57.490790 139906068133696 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:418: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0611 21:44:57.514949 139906068133696 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:499: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0611 21:44:57.855161 139906068133696 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFl

In [11]:
with orig_graph.as_default():
    
    with tf.compat.v1.variable_scope("loss", reuse=tf.compat.v1.AUTO_REUSE):
        output_layer = po  # tf.nn.dropout(po, keep_prob=0.5)
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)

        one_hot_labels = tf.one_hot(tf.constant(labels), depth=2, dtype=tf.float32)

        y_predictions = tf.argmax(logits, axis=-1)
#         if not False:  # self.multilabel:
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        y_probas = tf.nn.softmax(logits, axis=-1)
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
#         else:
#             y_probas = tf.nn.sigmoid(logits)
#             loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=one_hot_labels, logits=logits))

In [12]:
orig_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bi

In [13]:
with orig_graph.as_default():
    from tensorflow.python.ops import variables
    orig_vs = variables._all_saveable_objects()
    orig_saver = tf.compat.v1.train.Saver(orig_vs)
    orig_saver_checkpoint_path = 'models/paraphraser_rubert/model_rubert'  # 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
     # 'models/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
    orig_saver.restore(orig_sess, orig_saver_checkpoint_path)

# Checkpoint exploration

In [14]:
checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
# 'models/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
var_names = tf.train.list_variables(checkpoint_path)
checkpoint_reader = tf.train.load_checkpoint(checkpoint_path)

In [15]:
checkpoint_vars = set()
for v, _ in var_names:
    checkpoint_vars.add(v)

In [16]:
checkpoint_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer_1',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 

In [17]:
len(checkpoint_vars)

209

In [18]:
bert_vars = set()
for v in b_.variables:
    bert_vars.add(v.name.split(':')[0])

In [19]:
bert_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel

In [20]:
len(bert_vars)

199

# General equality testing

In [50]:
for i in range(len(orig_graph.get_collection('trainable_variables'))):
    with orig_graph.as_default():
        orig_v = orig_sess.run(orig_graph.get_collection('trainable_variables')[i].read_value())
    with keras_graph.as_default():
        keras_v = keras_sess.run(keras_graph.get_collection('trainable_variables')[i].read_value())
    print(keras_graph.get_collection('trainable_variables')[i])
    try:
        np.testing.assert_allclose(keras_v, orig_v)
    except:
        print("LayerNorm variables orders are different")

<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>
<tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>
<tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>
<tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/at

<tf.Variable 'bert/encoder/layer_4/intermediate/dense/bias:0' shape=(3072,) dtype=float32>
<tf.Variable 'bert/encoder/layer_4/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_4/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_4/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_4/output/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_5/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/key/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
<

<tf.Variable 'bert/encoder/layer_9/intermediate/dense/kernel:0' shape=(768, 3072) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/intermediate/dense/bias:0' shape=(3072,) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_9/output/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_10/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/key/bias:0' shape=(768,) dtype=float32

check at least emb_layer_norm weight are ok

In [22]:
with keras_graph.as_default():
    keras_gamma = keras_sess.run(keras_graph.get_collection('trainable_variables')[3].read_value())
    keras_beta = keras_sess.run(keras_graph.get_collection('trainable_variables')[4].read_value())
with orig_graph.as_default():
    orig_gamma = orig_sess.run(orig_graph.get_collection('trainable_variables')[4].read_value())
    orig_beta = orig_sess.run(orig_graph.get_collection('trainable_variables')[3].read_value())
np.testing.assert_allclose(keras_gamma, orig_gamma)
np.testing.assert_allclose(keras_beta, orig_beta)

In [23]:
current_mask = input_mask

In [24]:
with keras_graph.as_default():
    
    emb_output_op_ = b_.embed_dropout(
        b_.embed_layer_norm(
            b_.embed(input_ids)
        ), training=False
    )
    all_encoder_layers_op_ = [b_.layers[3].layers[0](emb_output_op_, mask=tf.constant(current_mask))]
    for i in range(1, 12):
        all_encoder_layers_op_.append(b_.layers[3].layers[i](all_encoder_layers_op_[i-1], mask=tf.constant(current_mask)))
    emb_output_ = keras_sess.run(emb_output_op_)
    all_encoder_layers_ = keras_sess.run(all_encoder_layers_op_)
    pooled_output_ = keras_sess.run(b_(input_ids))

In [25]:
with orig_graph.as_default():
    emb_output = orig_sess.run(b.get_embedding_output(), feed_dict={input_ids_ph: input_ids,
                                                                    input_masks_ph: current_mask,
                                                                    token_types_ph: token_types})
    all_encoder_layers = orig_sess.run(b.get_all_encoder_layers(), feed_dict={input_ids_ph: input_ids,
                                                                              input_masks_ph: current_mask,
                                                                              token_types_ph: token_types})
    pooled_output = orig_sess.run(b.get_pooled_output(), feed_dict={input_ids_ph: input_ids,
                                                                    input_masks_ph: current_mask,
                                                                    token_types_ph: token_types})

In [26]:
np.testing.assert_allclose(emb_output_, emb_output)

In [27]:
for i in range(12):
    np.testing.assert_allclose(all_encoder_layers_[i], all_encoder_layers[i])

In [28]:
np.testing.assert_allclose(pooled_output_, pooled_output)

# Paraphraser testing

In [29]:
from bert_dp.metrics import SparseBinaryF1Score

In [30]:
with keras_graph.as_default():
    paraphrase_detector = tf.keras.Sequential([b_,
                                           tf.keras.layers.Dropout(rate=0.5),
                                           tf.keras.layers.Dense(2)])
#     paraphrase_detector.build(input_shape=(BATCH_SIZE, None))
    loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    paraphrase_detector.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-05,
                                                                   epsilon=1e-6),
                                loss=loss_obj,
                                metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                                         SparseBinaryF1Score()])
    paraphrase_detector(input_ids)

In [31]:
keras_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32>,
 <tf.

In [32]:
with keras_graph.as_default():
    w_trained_, b_trained = keras_sess.run([keras_graph.get_collection('trainable_variables')[-4].read_value(),
                                            keras_graph.get_collection('trainable_variables')[-3].read_value()])

In [35]:
with keras_graph.as_default():
    r = paraphrase_detector.layers[-1].set_weights([w_trained_.transpose(), b_trained])

In [21]:
with keras_graph.as_default():
    paraphrase_detector.save_weights(filepath='models/paraphraser_rubert_converted/model_rubert')

In [42]:
with keras_graph.as_default():
    for v in paraphrase_detector.layers[-1].weights:
        keras_sess.run(v.initializer)

In [43]:
with keras_graph.as_default():
    w, b = paraphrase_detector.layers[-1].get_weights()

In [44]:
np.testing.assert_allclose(w, w_trained_.transpose())
np.testing.assert_allclose(b, b_trained)

In [45]:
with keras_graph.as_default():
    logits_value_ = keras_sess.run(paraphrase_detector(input_ids, training=False))
    loss_value_ = keras_sess.run(loss_obj(labels, logits_value_))

In [46]:
with orig_graph.as_default():
    logits_value = orig_sess.run(logits, feed_dict={input_ids_ph: input_ids,
                                                    input_masks_ph: current_mask,
                                                    token_types_ph: token_types})
    loss_value = orig_sess.run(loss, feed_dict={input_ids_ph: input_ids,
                                                input_masks_ph: current_mask,
                                                token_types_ph: token_types})

In [51]:
np.testing.assert_allclose(logits_value_, logits_value)
# np.testing.assert_allclose(loss_value_, loss_value)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(mismatch 100.0%)
 x: array([[-0.702521,  0.496038],
       [-0.714776,  0.63495 ],
       [-0.702521,  0.496038],
       [-0.714776,  0.63495 ]], dtype=float32)
 y: array([[-0.361206,  0.498325],
       [-0.699516,  0.998533],
       [-0.361206,  0.498325],
       [-0.699516,  0.998533]], dtype=float32)

# Debugging stuff

In [22]:
from bert_dp.modeling import attention_layer, layer_norm, create_attention_mask_from_input_mask, dropout

In [23]:
from bert_dp.modeling import get_shape_list, reshape_from_matrix, reshape_to_matrix, create_initializer

In [24]:
from bert_dp.activations import gelu

In [25]:
current_mask = input_mask

In [26]:
with orig_graph.as_default():
    msk = orig_sess.run(tf.expand_dims(
        create_attention_mask_from_input_mask(tf.constant(emb_output),
                                              tf.constant(current_mask)),
        axis=[1])
    )

In [27]:
msk.shape

(4, 1, 512, 512)

In [28]:
with keras_graph.as_default():
    msk_ = keras_sess.run(bm_.create_self_attention_mask_from_input_mask(
#         tf.constant(emb_output),
        tf.constant(current_mask)
    )
                       )

In [29]:
msk_.shape

(4, 1, 1, 512)

In [30]:
np.testing.assert_allclose(msk_, msk[:, :, 1:2, :])

In [31]:
g = tf.compat.v1.Graph()

In [32]:
with g.as_default():
    sess = tf.compat.v1.Session()

In [33]:
import math

In [34]:
with g.as_default():
    reuse = tf.compat.v1.AUTO_REUSE
    with tf.compat.v1.variable_scope("bert/encoder/layer_0/attention", reuse=reuse):
        with tf.compat.v1.variable_scope("self", reuse=None):
#             attention_head = attention_layer(
            from_tensor=tf.constant(emb_output)  # ,
            to_tensor=tf.constant(emb_output)  # ,
            attention_mask=create_attention_mask_from_input_mask(from_tensor=tf.constant(emb_output),
                                                                 to_mask=tf.constant(current_mask))  # ,
            num_attention_heads=12  # ,
            size_per_head=(768 // 12)  # ,
            query_act = None,
            key_act = None,
            value_act = None
            attention_probs_dropout_prob=0.0  # , # !
            initializer_range=0.02  # ,
            do_return_2d_tensor=False  # ,
            batch_size=BATCH_SIZE  # ,
            from_seq_length=MAX_LEN  # ,
            to_seq_length=MAX_LEN
#             )
            def transpose_for_scores(input_tensor, batch_size, num_attention_heads,
                           seq_length, width):
                output_tensor = tf.reshape(
                    input_tensor, [batch_size, seq_length, num_attention_heads, width])

                output_tensor = tf.transpose(output_tensor, [0, 2, 1, 3])
                return output_tensor

            from_shape = get_shape_list(from_tensor, expected_rank=[2, 3], name='from')
            to_shape = get_shape_list(to_tensor, expected_rank=[2, 3], name='to')

            if len(from_shape) != len(to_shape):
                raise ValueError("The rank of `from_tensor` must match the rank of `to_tensor`.")

            if len(from_shape) == 3:
                batch_size = from_shape[0]
                from_seq_length = from_shape[1]
                to_seq_length = to_shape[1]
            elif len(from_shape) == 2:
                if (batch_size is None or from_seq_length is None or to_seq_length is None):
                    raise ValueError(
                        "When passing in rank 2 tensors to attention_layer, the values "
                        "for `batch_size`, `from_seq_length`, and `to_seq_length` "
                        "must all be specified.")

            # Scalar dimensions referenced here:
            #   B = batch size (number of sequences)
            #   F = `from_tensor` sequence length
            #   T = `to_tensor` sequence length
            #   N = `num_attention_heads`
            #   H = `size_per_head`

            from_tensor_2d = reshape_to_matrix(from_tensor)
            to_tensor_2d = reshape_to_matrix(to_tensor)

            # `query_layer` = [B*F, N*H]
            query_layer_ = tf.layers.dense(
              from_tensor_2d,
              num_attention_heads * size_per_head,
#               activation=query_act,
              name="query",
              kernel_initializer=create_initializer(initializer_range))

            # `key_layer` = [B*T, N*H]
            key_layer_ = tf.layers.dense(
              to_tensor_2d,
              num_attention_heads * size_per_head,
#               activation=key_act,
              name="key",
              kernel_initializer=create_initializer(initializer_range))

            # `value_layer` = [B*T, N*H]
            value_layer_ = tf.layers.dense(
              to_tensor_2d,
              num_attention_heads * size_per_head,
#               activation=value_act,
              name="value",
              kernel_initializer=create_initializer(initializer_range))

            # `query_layer` = [B, N, F, H]
            query_layer = transpose_for_scores(query_layer_, batch_size,
                                             num_attention_heads, from_seq_length,
                                             size_per_head)

            # `key_layer` = [B, N, T, H]
            key_layer = transpose_for_scores(key_layer_, batch_size, num_attention_heads,
                                           to_seq_length, size_per_head)

            # Take the dot product between "query" and "key" to get the raw
            # attention scores.
            # `attention_scores` = [B, N, F, T]
            raw_attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
            raw_attention_scores = tf.multiply(raw_attention_scores,
                                         1.0 / math.sqrt(float(size_per_head)))

            if attention_mask is not None:
                # `attention_mask` = [B, 1, F, T]
                attention_mask = tf.expand_dims(attention_mask, axis=[1])

                # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
                # masked positions, this operation will create a tensor which is 0.0 for
                # positions we want to attend and -10000.0 for masked positions.
                adder = (1.0 - tf.cast(attention_mask, tf.float32)) * -10000.0

                # Since we are adding it to the raw scores before the softmax, this is
                # effectively the same as removing these entirely.
                attention_scores = raw_attention_scores + adder

            # Normalize the attention scores to probabilities.
            # `attention_probs` = [B, N, F, T]
            attention_probs = tf.nn.softmax(attention_scores)

            # This is actually dropping out entire tokens to attend to, which might
            # seem a bit unusual, but is taken from the original Transformer paper.
#             attention_probs = dropout(attention_probs, attention_probs_dropout_prob)

            # `value_layer` = [B, T, N, H]
            value_layer = tf.reshape(
              value_layer_,
              [batch_size, to_seq_length, num_attention_heads, size_per_head])

            # `value_layer` = [B, N, T, H]
            value_layer = tf.transpose(value_layer, [0, 2, 1, 3])

            # `context_layer` = [B, N, F, H]
            context_layer_ = tf.matmul(attention_probs, value_layer)

            # `context_layer` = [B, F, N, H]
            context_layer = tf.transpose(context_layer_, [0, 2, 1, 3])

            if do_return_2d_tensor:
                # `context_layer` = [B*F, N*H]
                attention_head = tf.reshape(
                    context_layer,
                    [batch_size * from_seq_length, num_attention_heads * size_per_head])
            else:
                # `context_layer` = [B, F, N*H]
                attention_head = tf.reshape(
                    context_layer,
                    [batch_size, from_seq_length, num_attention_heads * size_per_head])
            
        with tf.compat.v1.variable_scope("output", reuse=None):
            attention_output = tf.layers.dense(
                      attention_head,
                      768,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

#             attention_output = tf.layers.dropout(attention_output, 0.1)
            attention_output = layer_norm(attention_output + tf.constant(emb_output))

    # The activation is only applied to the "intermediate" hidden layer.
    with tf.variable_scope("bert/encoder/layer_0/intermediate", reuse=reuse):
        intermediate_output = tf.layers.dense(attention_output,
                                              3072,
                                              activation=gelu,
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

    # Down-project back to `hidden_size` then add the residual.
    with tf.variable_scope("bert/encoder/layer_0/output", reuse=reuse):
        layer_output = tf.layers.dense(
            intermediate_output,
            768,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
#         layer_output = tf.layers.dropout(layer_output, 0.1)
        layer_output = layer_norm(layer_output + attention_output)

In [35]:
g.get_collection('trainable_variables')

[<tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/LayerNorm/gamma:0' shape=(768,) dty

In [36]:
with g.as_default():
    saver = tf.compat.v1.train.Saver(variables._all_saveable_objects())
    saver_checkpoint_path = 'models/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
    # 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
    saver.restore(sess, saver_checkpoint_path)

In [37]:
with g.as_default():
    q_out = sess.run(query_layer_)
    k_out = sess.run(key_layer_)
    v_out = sess.run(value_layer_)
    qout = sess.run(query_layer)
    kout = sess.run(key_layer)
    vout = sess.run(value_layer)
with keras_graph.as_default():
    q_out_ = keras_sess.run(#first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wq(emb_output),
#         batch_size=BATCH_SIZE
#     )
                          )
    k_out_ = keras_sess.run(#first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wk(emb_output),
#         batch_size=BATCH_SIZE
#     )
                          )
    v_out_ = keras_sess.run(#first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wv(emb_output),
#         batch_size=BATCH_SIZE
#     )
                          )
    qout_ = keras_sess.run(first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wq(emb_output),
        batch_size=BATCH_SIZE
    )
                          )
    kout_ = keras_sess.run(first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wk(emb_output),
        batch_size=BATCH_SIZE
    )
                          )
    vout_ = keras_sess.run(first_transformer_block.mhsa.split_heads(
        first_transformer_block.mhsa.wv(emb_output),
        batch_size=BATCH_SIZE
    )
                          )

In [38]:
np.testing.assert_allclose(q_out_.reshape(-1, 768), q_out)
np.testing.assert_allclose(k_out_.reshape(-1, 768), k_out)
np.testing.assert_allclose(v_out_.reshape(-1, 768), v_out)
np.testing.assert_allclose(qout_, qout)
np.testing.assert_allclose(kout_, kout)
np.testing.assert_allclose(vout_, vout)

In [39]:
from bert_dp.attention import scaled_dot_product_attention

In [40]:
with g.as_default():
    raw_a_scores = sess.run(raw_attention_scores)
    a_scores = sess.run(attention_scores)
    a_probs = sess.run(attention_probs)
    a_out = sess.run(context_layer_)
with keras_graph.as_default():
    raw_attention_scores_ = tf.matmul(qout_, kout_, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    raw_attention_scores_ = raw_attention_scores_ / tf.math.sqrt(64.0)
    raw_a_scores_ = keras_sess.run(raw_attention_scores_)
    attention_mask_ = bm_.create_self_attention_mask_from_input_mask(current_mask)
    a_scores_ = keras_sess.run(raw_attention_scores_ + ((1.0 - attention_mask_) * -10000.0))
    a_probs_ = keras_sess.run(tf.nn.softmax(a_scores_))  # (..., seq_len_q, seq_len_k)
    a_out_ = keras_sess.run(tf.matmul(a_probs_, vout_))

In [41]:
np.testing.assert_allclose(raw_a_scores_, raw_a_scores)
np.testing.assert_allclose(a_scores_, a_scores)
np.testing.assert_allclose(a_probs_, a_probs, atol=1e-6)
np.testing.assert_allclose(a_out_, a_out, atol=1e-5)

In [42]:
with g.as_default():
    mhsa = sess.run(attention_head)
    attention_output_res = sess.run(attention_output)
    intermediate_output_res = sess.run(intermediate_output)
    layer_output_res = sess.run(layer_output)
with keras_graph.as_default():
    mhsa_ = keras_sess.run(first_transformer_block.mhsa(emb_output, emb_output, emb_output,
                                                        mask=bm_.create_self_attention_mask_from_input_mask(current_mask)
                                                       )
                          )

    attention_output_ = first_transformer_block.layernorm1(
        emb_output + first_transformer_block.dropout1(
            first_transformer_block.dense(mhsa), training=False
        )
    )
    
    attention_output_res_ = keras_sess.run(attention_output_)

    intermediate_output_res_ = keras_sess.run(first_transformer_block.pff1(attention_output_res_))
    
    layer_output_ = first_transformer_block.layernorm2(
        attention_output_res_ + first_transformer_block.dropout2(
            first_transformer_block.pff2(intermediate_output_res_), training=False
        )
    )
    layer_output_res_ = keras_sess.run(layer_output_)

In [43]:
np.testing.assert_allclose(mhsa_, mhsa)

In [44]:
np.testing.assert_allclose(attention_output_res_, attention_output_res)

In [45]:
np.testing.assert_allclose(intermediate_output_res_, intermediate_output_res)

In [46]:
np.testing.assert_allclose(layer_output_res_, layer_output_res)